# USAC: Deeper Dive

**Amaç:** OpenCV'dekı USAC implementasyonunun homografi tahminiyle alakalı kısmını daha iyi anlamak.

**Alt amaçlar:**

- Params kullanarak findHomography fonksiyonunu çağırmakta uzmanlaşmak. 
- cv.USAC_MAGSAC, cv.USAC_ACCURATE gibi yöntemleri sadece params kullanarak elde edebilmek.
- Hangi params kombinasyonlarının mantıklı olduğunu anlamak.
- Genel olarak frameworkü anlamak ve özel olarak bileşenlerin nasıl çalıştığını kabaca anlamak.

In [1]:
# cv.findHomography(srcPoints, dstPoints, params)

# USAC: https://docs.opencv.org/4.x/d0/d74/md__build_4_x-contrib_docs-lin64_opencv_doc_tutorials_calib3d_usac.html

# Params: https://docs.opencv.org/4.x/d6/dd0/structcv_1_1UsacParams.html

'''
double 	confidence
 
PolishingMethod 	final_polisher
 
int 	final_polisher_iterations
 
bool 	isParallel
 
int 	loIterations -- Default: 10
    Number of iterations for Local Optimization method.
    By increasing loIterations the output model could be more accurate, however, the computationial time may also increase.
 
LocalOptimMethod 	loMethod
 
int 	loSampleSize -- Default: 14
    Maximum sample number for Local Optimization.
    Note, that by increasing loSampleSize the accuracy of model can increase as well as the computational time. 
    However, it is recommended to keep value less than 100, because estimation on low number of points is faster and more robust.
 
int 	maxIterations
 
NeighborSearchMethod 	neighborsSearch
 
int 	randomGeneratorState
 
SamplingMethod 	sampler
 
ScoreMethod 	score
 
double 	threshold
'''

# PolishingMethod
# "... in the end the final so-far-the-best model is polished by non minimal estimation of all found inliers."
# Bunda default falan ne ki?
'''
cv.NONE_POLISHER
cv.LSQ_POLISHER
cv.MAGSAC
cv.COV_POLISHER
'''

# LocalOptimMethod
'''
cv.LOCAL_OPTIM_NULL
cv.LOCAL_OPTIM_INNER_LO (Bu veya alttaki default??) -- Bu degilmis.
cv.LOCAL_OPTIM_INNER_AND_ITER_LO  (Bu veya ustteki default??) -- Buymus.
cv.LOCAL_OPTIM_GC (quite precise however computationally slower)
cv.LOCAL_OPTIM_SIGMA (MAGSAC'in yontemi. MAGSAC score ile beraber kullanmak mantikli.)
'''

# NeighborSearchMethod
# NEIGH_FLANN_RADIUS and NEIGH_FLANN_RADIUS are not able to PnP solver, since there are 3D object points.
# For NAPSAC, Progressive NAPSAC or Graph-Cut methods is required to build a neighborhood graph. In framework there are 3 options to do it:
'''
cv.NEIGH_FLANN_KNN
    Estimate neighborhood graph using OpenCV FLANN K nearest-neighbors. The default value for KNN is 7. KNN method may work good for sampling but not good for GC-RANSAC.
cv.NEIGH_FLANN_RADIUS
    Similarly as in previous case finds neighbor points which distance is less than 20 pixels.
cv.NEIGH_GRID
    For finding points' neighborhood tiles points in cells using hash-table. The method is described in [2]. Less accurate than NEIGH_FLANN_RADIUS, although significantly faster.
    [2] Daniel Barath, Maksym Ivashechkin, and Jiri Matas. 2019. Progressive NAPSAC: Sampling from gradually growing neighborhoods. arXiv preprint arXiv:1906.02295.

    TODO: Bu hash-table fikri baska yerlerde de kullanilabilir.

'''

# SamplingMethod
'''
cv.SAMPLING_UNIFORM (default)
cv.SAMPLING_PROGRESSIVE_NAPSAC
cv.SAMPLING_NAPSAC
cv.SAMPLING_PROSAC
'''

# ScoreMethod
'''
cv.SCORE_METHOD_RANSAC
cv.SCORE_METHOD_MSAC (default)
cv.SCORE_METHOD_MAGSAC
cv.SCORE_METHOD_LMEDS
'''

'''
Error metric which describes error distance of point to estimated model.
Re-projection distance - used for affine, homography and projection matrices. For homography also symmetric re-projection distance can be used.
Sampson distance - used for Fundamental matrix.
Symmetric Geometric distance - used for Essential matrix.

Symmetric re-projection distance nasil secilebiliyor?? 
'''

# USAC makalesine bak.


'\nError metric which describes error distance of point to estimated model.\nRe-projection distance - used for affine, homography and projection matrices. For homography also symmetric re-projection distance can be used.\nSampson distance - used for Fundamental matrix.\nSymmetric Geometric distance - used for Essential matrix.\n\nSymmetric re-projection distance nasil secilebiliyor?? \n'

In [2]:
'''
void setParameters (int flag, Ptr<Model> &params, EstimationMethod estimator, double thr,
        int max_iters, double conf, bool mask_needed) {
    switch (flag) {
        case USAC_DEFAULT:
            params = Model::create(thr, estimator, SamplingMethod::SAMPLING_UNIFORM, conf, max_iters, ScoreMethod::SCORE_METHOD_MSAC);
            params->setLocalOptimization(LocalOptimMethod ::LOCAL_OPTIM_INNER_AND_ITER_LO);
            break;
        case USAC_MAGSAC:
            params = Model::create(thr, estimator, SamplingMethod::SAMPLING_UNIFORM, conf, max_iters, ScoreMethod::SCORE_METHOD_MAGSAC);
            params->setLocalOptimization(LocalOptimMethod ::LOCAL_OPTIM_SIGMA);
            params->setLOSampleSize(params->isHomography() ? 75 : 50);
            params->setLOIterations(params->isHomography() ? 15 : 10);
            break;
        case USAC_PARALLEL:
            params = Model::create(thr, estimator, SamplingMethod::SAMPLING_UNIFORM, conf, max_iters, ScoreMethod::SCORE_METHOD_MSAC);
            params->setParallel(true);
            params->setLocalOptimization(LocalOptimMethod ::LOCAL_OPTIM_INNER_LO);
            break;
        case USAC_ACCURATE:
            params = Model::create(thr, estimator, SamplingMethod::SAMPLING_UNIFORM, conf, max_iters, ScoreMethod::SCORE_METHOD_MSAC);
            params->setLocalOptimization(LocalOptimMethod ::LOCAL_OPTIM_GC);
            params->setLOSampleSize(20);
            params->setLOIterations(25);
            break;
        case USAC_FAST:
            params = Model::create(thr, estimator, SamplingMethod::SAMPLING_UNIFORM, conf, max_iters, ScoreMethod::SCORE_METHOD_MSAC);
            params->setLocalOptimization(LocalOptimMethod ::LOCAL_OPTIM_INNER_AND_ITER_LO);
            params->setLOIterations(5);
            params->setLOIterativeIters(3);
            break;
        case USAC_PROSAC:
            params = Model::create(thr, estimator, SamplingMethod::SAMPLING_PROSAC, conf, max_iters, ScoreMethod::SCORE_METHOD_MSAC);
            params->setLocalOptimization(LocalOptimMethod ::LOCAL_OPTIM_INNER_LO);
            break;
        case USAC_FM_8PTS:
            params = Model::create(thr, EstimationMethod::Fundamental8,SamplingMethod::SAMPLING_UNIFORM, conf, max_iters,ScoreMethod::SCORE_METHOD_MSAC);
            params->setLocalOptimization(LocalOptimMethod ::LOCAL_OPTIM_INNER_LO);
            break;
        default: CV_Error(cv::Error::StsBadFlag, "Incorrect flag for USAC!");
    }
    // do not do too many iterations for PnP
    if (estimator == EstimationMethod::P3P) {
        if (params->getLOInnerMaxIters() > 15)
            params->setLOIterations(15);
        params->setLOIterativeIters(0);
    }

    params->maskRequired(mask_needed);
}
'''

# Ozet bilgiler:
# Sadece homography icin.
# USAC_PROSAC haric hepsinde SAMPLING_UNIFORM.
# USAC_MAGSAC haric hepsinde SCORE_METHOD_MSAC.
# USAC_PARALLEL haric hicbiri paralel degil.
# LocalOptimMethod
#  USAC_MAGSAC -> LOCAL_OPTIM_SIGMA (loSampleSize: 75, loIterations: 15)
#  USAC_DEFAULT, USAC_FAST -> LOCAL_OPTIM_INNER_AND_ITER_LO (Default icin default degerler, FAST icin loIterations: 5, loIterativeIters: 3) ----- loIterativeIters ne ki?????
#  USAC_ACCURATE -> LOCAL_OPTIM_GC (loSampleSize: 20, loIterations: 25)
#  USAC_PARALLEL, USAC_PROSAC -> LOCAL_OPTIM_INNER_LO (Ikisi icin de default degerler)


'\nvoid setParameters (int flag, Ptr<Model> &params, EstimationMethod estimator, double thr,\n        int max_iters, double conf, bool mask_needed) {\n    switch (flag) {\n        case USAC_DEFAULT:\n            params = Model::create(thr, estimator, SamplingMethod::SAMPLING_UNIFORM, conf, max_iters, ScoreMethod::SCORE_METHOD_MSAC);\n            params->setLocalOptimization(LocalOptimMethod ::LOCAL_OPTIM_INNER_AND_ITER_LO);\n            break;\n        case USAC_MAGSAC:\n            params = Model::create(thr, estimator, SamplingMethod::SAMPLING_UNIFORM, conf, max_iters, ScoreMethod::SCORE_METHOD_MAGSAC);\n            params->setLocalOptimization(LocalOptimMethod ::LOCAL_OPTIM_SIGMA);\n            params->setLOSampleSize(params->isHomography() ? 75 : 50);\n            params->setLOIterations(params->isHomography() ? 15 : 10);\n            break;\n        case USAC_PARALLEL:\n            params = Model::create(thr, estimator, SamplingMethod::SAMPLING_UNIFORM, conf, max_iters, ScoreMetho

In [3]:
# https://github.com/opencv/opencv/blob/4.x/modules/calib3d/src/usac/ransac_solvers.cpp

'''
UsacParams::UsacParams() {
    confidence=0.99;
    isParallel=false;
    loIterations=5;
    loMethod=LOCAL_OPTIM_INNER_LO;
    loSampleSize=14;
    maxIterations=5000;
    neighborsSearch=NEIGH_GRID;
    randomGeneratorState=0;
    sampler=SAMPLING_UNIFORM;
    score=SCORE_METHOD_MSAC;
    threshold=1.5;
    final_polisher=COV_POLISHER;
    final_polisher_iterations=3;
}
'''

# Dokumantasyonda belirtilen default degerler burada gecerli degil. (Mesela loIterations 10 deniyor ama burada 5.)

'\nUsacParams::UsacParams() {\n    confidence=0.99;\n    isParallel=false;\n    loIterations=5;\n    loMethod=LOCAL_OPTIM_INNER_LO;\n    loSampleSize=14;\n    maxIterations=5000;\n    neighborsSearch=NEIGH_GRID;\n    randomGeneratorState=0;\n    sampler=SAMPLING_UNIFORM;\n    score=SCORE_METHOD_MSAC;\n    threshold=1.5;\n    final_polisher=COV_POLISHER;\n    final_polisher_iterations=3;\n}\n'

In [4]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [5]:
from utils import *

In [6]:
img1_no = 1
img2_no = 4

img1_path = f'homography_dataset/img{img1_no}.png'
img2_path = f'homography_dataset/img{img2_no}.png'
h_path = f'homography_dataset/H{img1_no}to{img2_no}p'

img1 = read_image(img1_path, is_grayscale=True)
img2 = read_image(img2_path, is_grayscale=True)

kp1 = detect_keypoints(img1)
kp2 = detect_keypoints(img2)

des1 = compute_descriptors(img1, kp1)
des2 = compute_descriptors(img2, kp2)

bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
matches = bf.match(des1, des2)

pts1 = np.int32([kp1[m.queryIdx].pt for m in matches])
pts2 = np.int32([kp2[m.trainIdx].pt for m in matches])

In [7]:
#H, mask = cv.findHomography(pts1, pts2, cv.USAC_DEFAULT)

params = cv.UsacParams()
# params.confidence = ...  # float
# params.isParallel = ...  # bool
params.loIterations = 5  # int
# params.loMethod = ...  # LocalOptimMethod
# params.loSampleSize = ...  # int
# params.maxIterations = ...  # int
# params.neighborsSearch = ...  # NeighborSearchMethod
# params.randomGeneratorState = ...  # int
# params.sampler = ...  # SamplingMethod
# params.score = ...  # ScoreMethod
# params.threshold = ...  # float
# params.final_polisher = ...  # PolishingMethod
# params.final_polisher_iterations = ...  # int
H, mask = cv.findHomography(pts1, pts2, params=params)

mask = mask.ravel()
inlier_pts1 = pts1[mask == 1]
inlier_pts2 = pts2[mask == 1]

H_true = np.loadtxt(h_path)
print('Average corner error:', round(average_corner_error(*img1.shape, H_true, H), 3))
# Genel olarak 10'un altinda olmali.

'''
img1 - img4

ratio=1.0, cross_check=False

cv.RANSAC 6.235
cv.USAC_MAGSAC 1.678
cv.USAC_ACCURATE 1.173

ratio=1.0, cross_check=True

cv.RANSAC 2.886
cv.USAC_MAGSAC 2.362
cv.USAC_ACCURATE 1.029
cv.USAC_DEFAULT 1.016

ratio=0.7, cross_check=False

cv.RANSAC 2.131
cv.USAC_MAGSAC 3.955
cv.USAC_ACCURATE 2.302
'''



# cv.USAC_DEFAULT vermeyle cv.UsacParams() vermek farkli sonuc. Yani bu ikisi farkli seyler!

# Alttakiler hep img1-img4, ratio=1.0, cross_check=True.

'''
cv.UsacParams() 
1.868

params.loIterations = 10
1.857
Neden usttekiyle ayni degil?

Evet 5 degeri orijinaliyle ayni sonucu verdi.

'''

Average corner error: 1.788


'\ncv.UsacParams() \n1.868\n\nparams.loIterations = 10\n1.857\nNeden usttekiyle ayni degil?\n\nEvet 5 degeri orijinaliyle ayni sonucu verdi.\n\n'